<a href="https://colab.research.google.com/github/omarjilanijidan/Iris-Cataract-Detection-From-Fundus-Images/blob/main/VGG19_with_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
import os
import numpy as np
import cv2
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model

In [ ]:
# Authenticate and mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
image_size = 224
dataset_dir = ""


In [ ]:
def create_dataset(image_category, label):
    dataset = []
    labels = []
    image_folder = os.path.join(dataset_dir, image_category)
    for img in tqdm(os.listdir(image_folder)[:1000]):
        image_path = os.path.join(image_folder, img)
        try:
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            if image is None:
                raise Exception("Failed to load image data")
            image = cv2.resize(image, (image_size, image_size))
            image = image / 255.0
            dataset.append(image)
            labels.append(label)
        except Exception as e:
            print(e)
    return np.array(dataset), np.array(labels)

In [ ]:
cataract_images, cataract_labels = create_dataset("/content/drive/MyDrive/Papers/Dataset Fundus Image/Cataract", 1)
normal_images, normal_labels = create_dataset("/content/drive/MyDrive/Papers/Dataset Fundus Image/NDG", 0)

100%|██████████| 1000/1000 [00:08<00:00, 114.32it/s]


In [ ]:
X = np.concatenate((cataract_images, normal_images[:1000]), axis=0)
y = np.concatenate((cataract_labels, normal_labels[:1000]), axis=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Print number of images used for training and testing
print(f"Number of images used for training: {len(X_train)}")
print(f"Number of images used for testing: {len(X_test)}")


Number of images used for training: 1600
Number of images used for testing: 400


In [ ]:
# Pre-augmentation
augmented_images = []
augmented_labels = []
#Augmentation training data
for image, label in zip(X_train, y_train):
    # Random rotation
    random_rotation = random.uniform(-20, 20)
    rotated_image = cv2.getRotationMatrix2D((image_size / 2, image_size / 2), random_rotation, 1)
    rotated_image = cv2.warpAffine(image, rotated_image, (image_size, image_size))
    augmented_images.append(rotated_image)
    augmented_labels.append(label)

    # Random horizontal flip
    if random.random() > 0.5:
        flipped_image = cv2.flip(image, 1)
        augmented_images.append(flipped_image)
        augmented_labels.append(label)

In [ ]:
# Concatenate augmented images and labels with original data
X_train = np.concatenate((X_train, np.array(augmented_images)), axis=0)
y_train = np.concatenate((y_train, np.array(augmented_labels)), axis=0)

In [ ]:
# Print number of images used for training and testing
print(f"Number of images used for training: {len(X_train)}")
print(f"Number of images used for testing: {len(X_test)}")


Number of images used for training: 3997
Number of images used for testing: 400


In [ ]:
# Create VGG19 base model
vgg19 = VGG19(include_top=False, weights="imagenet", input_shape=(image_size, image_size, 3))

80134624/80134624 [==============================] - 1s 0us/step


In [ ]:
# Freeze base model layers
for layer in vgg19.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D
# Create custom model with VGG19 base and fully connected layers
inputs = vgg19.input
outputs = vgg19.output
outputs = Dropout(0.5)(outputs)
outputs = Dense(512, activation="relu")(outputs)
outputs = Dropout(0.5)(outputs)
outputs = Dense(49, activation="sigmoid")(outputs)
outputs = GlobalAveragePooling2D()(outputs)
predictions = Dense(1, activation="sigmoid")(outputs)
model = Model(inputs=inputs, outputs=predictions)

In [ ]:
# Compile model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
# Fit model on train data
history = model.fit(X_train, y_train, epochs=10, batch_size=49, validation_data=(X_test, y_test))

Epoch 1/10
82/82 [==============================] - 1401s 17s/step - loss: 0.3411 - accuracy: 0.8899 - val_loss: 0.1805 - val_accuracy: 0.9425
Epoch 2/10
82/82 [==============================] - 1398s 17s/step - loss: 0.1811 - accuracy: 0.9437 - val_loss: 0.1434 - val_accuracy: 0.9425
Epoch 3/10
82/82 [==============================] - 1399s 17s/step - loss: 0.1493 - accuracy: 0.9515 - val_loss: 0.1250 - val_accuracy: 0.9700
Epoch 4/10
82/82 [==============================] - 1393s 17s/step - loss: 0.1412 - accuracy: 0.9490 - val_loss: 0.1191 - val_accuracy: 0.9700
Epoch 5/10
82/82 [==============================] - 1390s 17s/step - loss: 0.1318 - accuracy: 0.9532 - val_loss: 0.1161 - val_accuracy: 0.9525
Epoch 6/10
82/82 [==============================] - 1392s 17s/step - loss: 0.1223 - accuracy: 0.9560 - val_loss: 0.1139 - val_accuracy: 0.9575
Epoch 7/10
82/82 [==============================] - 1392s 17s/step - loss: 0.1243 - accuracy: 0.9527 - val_loss: 0.1087 - val_accuracy: 0.9675